In [2]:
import requests
import xml.etree.ElementTree
import socket
import os

## Obtain Token

https://wiki.earthdata.nasa.gov/display/CMR/Creating+a+Token+Common

In [2]:
username = ""
password = ""
url = 'https://cmr.earthdata.nasa.gov/legacy-services/rest/tokens'

In [3]:
def get_ip():
    s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    try:
        s.connect(('8.8.8.8', 80))
        ip = s.getsockname()[0]
    except:
        ip = '127.0.0.1'
    finally:
        s.close()
    return ip

In [4]:
ip = get_ip()
request = """
<token>
    <username>{username}</username>
    <password>{password}</password>
    <client_id>podpac</client_id>
    <user_ip_address>{ip}</user_ip_address>
</token>
""".format(username=username, password=password,ip=ip)
headers = {"Content-Type": "application/xml"}
r = requests.post(url, data=request, headers=headers)

In [5]:
tree = xml.etree.ElementTree.fromstring(r.text)
token = [element.text for element in tree.findall('id')][0]

In [6]:
token

'D76A634F-9083-5D1D-2DFF-FEB67AA851AA'

## Get Data

In [62]:
base_url = "https://n5eil01u.ecs.nsidc.org/egi/request"
# base_url = "https://n5eil02u.ecs.nsidc.org/egi/request"
version = "004"
start_time = "2015-03-30"
end_time = "2015-06-12"
bbox = "-70,42,-69,43"
response_format = "HDF-EOS"
# response_format = "GeoTIFF"
token = token  # from above

# product = "SPL4SMAU"
# data_layers = "/Analysis_Data/sm_surface_analysis"

# product = "SPL4SMGP"
# data_layers = "/Geophysical_Data/sm_surface"

# product = "SPL3SMP"
# data_layers = "/Soil_Moisture_Retrieval_Data/soil_moisture"

# product = "SPL3SMAP"  # No granule with this name
# data_layers = "/Soil_Moisture_Retrieval_Data/soil_moisture"

# product = "SPL3SMP"
# data_layers = "/Soil_Moisture_Retrieval_Data/soil_moisture"

# product = "SPL3SMP_E"  # No granule with this name
# data_layers = ""


# product = "SPL2SMAP_S"  # No granule with this name
# data_layers = ""


In [63]:
url = "{base_url}?short_name={product}&version={version}".format(base_url=base_url, product=product, version=version,) + \
      "&time={start_time},{end_time}&Coverage={data_layers}".format(start_time=start_time, end_time=end_time, data_layers=data_layers, ) + \
      "&Bbox={bbox}&format={response_format}&token={token}".format(bbox=bbox, response_format=response_format, token=token)

r = requests.get(url)

In [64]:
r.text

'<?xml version="1.0" encoding="UTF-8" standalone="yes"?>\n<eesi:Exception xsi:schemaLocation="http://eosdis.nasa.gov/esi/rsp/e https://newsroom.gsfc.nasa.gov/esi/8.1/schemas/ESIAgentResponseExternal.xsd" xmlns="" xmlns:iesi="http://eosdis.nasa.gov/esi/rsp/i" xmlns:ssw="http://newsroom.gsfc.nasa.gov/esi/rsp/ssw" xmlns:eesi="http://eosdis.nasa.gov/esi/rsp/e" xmlns:esi="http://eosdis.nasa.gov/esi/rsp" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">\n    <Code>InternalError</Code>\n    <Message>No granules returned by CMR: https://cmr.earthdata.nasa.gov/search/granules?provider=NSIDC_ECS&amp;short_name=SPL3SMP_E&amp;version=004&amp;temporal=2015-03-30,2015-06-12&amp;token=D76A634F-9083-5D1D-2DFF-FEB67AA851AA</Message>\n</eesi:Exception>\n'

In [73]:
from datetime import datetime
s = datetime.now()
url = 'https://n5eil01u.ecs.nsidc.org/egi/request?short_name=SPL4SMGP&Coverage=/Geophysical_Data/sm_surface,/cell_lat,/cell_lon&format=HDF-EOS&version=004&time=2015-07-06T00:00:00,2015-07-08T00:00:00&Bbox=-82.0,38.0,-81.0,39.0&token=5DEEC6C2-8092-628B-1CE4-2A97C8FE2CF7'
r = requests.get(url)
e = datetime.now()
print(e-s)

0:00:14.644087


In [74]:
with open('output.zip', 'wb') as f:
    f.write(r.content)

In [10]:
f = h5py.File('output/139973806/SMAP_L4_SM_gph_20150705T223000_Vv4030_001_HEGOUT.h5')

In [11]:
[k for k in f]

['Geophysical_Data', 'Metadata', 'projection_information', 'time', 'x', 'y']

In [12]:
f = h5py.File('output 8/139973815/SMAP_L4_SM_aup_20150706T000000_Vv4030_001_HEGOUT.h5')

In [13]:
f['x'][()]

array([-7913576.47251037, -7904568.41753112, -7895560.36255187,
       -7886552.30757261, -7877544.25259336, -7868536.19761411,
       -7859528.14263485, -7850520.0876556 , -7841512.03267635,
       -7832503.97769709, -7823495.92271784])

In [14]:
f['y'][()]

array([4607620.47251037, 4598612.41753112, 4589604.36255187,
       4580596.30757261, 4571588.25259336, 4562580.19761411,
       4553572.14263485, 4544564.0876556 , 4535556.03267635,
       4526547.97769709, 4517539.92271784])

## Look at the Data

In [11]:
from podpac.data import Rasterio
import zipfile
from io import BytesIO
%matplotlib inline

In [17]:
# helper function to display all keys in hdf5 file
def explore(f, key):
    print(key)
    print([k for k in f[key]])
    if len(f[key]) and not isinstance(f[key], h5py.Dataset):
        for newkey in f[key]:
            explore(f, '{}/{}'.format(key, newkey))

    if isinstance(f[key], h5py.Dataset):
        print(key + ' attrs:')
        print([k for k in f[key].attrs])

In [56]:
with open('output.zip', 'wb') as f:
    f.write(r.content)

In [ ]:
f = BytesIO(r.content)
zip_file = zipfile.ZipFile(f)

for name in zip_file.namelist():
    h5data = zip_file.read(name)

In [19]:
with zipfile.ZipFile("output.zip","r") as zip_ref:
    zip_ref.extractall()

In [5]:
node = Rasterio(source='test.tiff')

In [6]:
node.native_coordinates

Coordinates
	lat: UniformCoordinates1d(lat): Bounds[4999469.950207469, 5089550.5], N[10], ctype['midpoint']
	lon: UniformCoordinates1d(lon): Bounds[-7143387.5, -7044298.8952282155], N[11], ctype['midpoint']

In [ ]:
o = node.eval(node.native_coordinates)
o.plot()

In [49]:
f = h5py.File('output 6/139973815/SMAP_L4_SM_aup_20150706T000000_Vv4030_001_HEGOUT.h5')

In [50]:
[k for k in f]

['Analysis_Data',
 'Forecast_Data',
 'Metadata',
 'Observations_Data',
 'cell_column',
 'cell_lat',
 'cell_lon',
 'cell_row',
 'time',
 'x',
 'y']

In [53]:
f = h5py.File('output 4/139973815/SMAP_L4_SM_aup_20150706T000000_Vv4030_001_HEGOUT.h5')

In [54]:
[k for k in f]

['Forecast_Data', 'Metadata', 'projection_information', 'time', 'x', 'y']

In [29]:
f = h5py.File('output 3/81805640/SMAP_L3_SM_AP_20150707_R13080_001_HEGOUT.h5')

## PODPAC Interface

In [2]:
from podpac.datalib import EGI
from podpac import settings

## EGI

In [ ]:
username = ""
password = ""
token = "DCF3DD10-C56C-1F7F-12FE-6408839955B8"

In [ ]:
node = EGI(token=token, short_name="SPL3SMA", version="003", data_key="/Soil_Moisture_Retrieval_Data/soil_moisture")

In [ ]:
node

In [ ]:
node.token_valid()

In [ ]:
node.get_token()

In [ ]:
node.token_valid()

In [ ]:
node.token

## SMAP

In [1]:
import podpac
from podpac.datalib.smap_egi import SMAP
import zipfile
import h5py

from podpac import Coordinates, clinspace
import numpy as np

import logging
logger = logging.getLogger('podpac')
logger.setLevel(logging.DEBUG)

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


In [2]:
c = Coordinates([clinspace(-82, -81, 10), clinspace(38, 39, 10), clinspace('2015-07-06', '2015-07-08', 10)], dims=['lon', 'lat', 'time'])
node = SMAP(product="SPL4SMAU")

In [3]:
%pdb
o = node.eval(c)

Automatic pdb calling has been turned ON


DEBUG:podpac.datalib.egi:EGI Token valid
DEBUG:podpac.datalib.egi:Querying EGI url: https://n5eil01u.ecs.nsidc.org/egi/request?short_name=SPL4SMAU&Coverage=/Analysis_Data/sm_surface_analysis,/x,/y&format=HDF-EOS&version=004&time=2015-07-06T00:00:00,2015-07-08T00:00:00&Bbox=-82.0,38.0,-81.0,39.0&token=056C40A1-E226-31F5-4746-F9B75A30679F
DEBUG:podpac.datalib.egi:Processing 10 EGI files from zip archive
DEBUG:podpac.datalib.egi:Reading file: 139973815/SMAP_L4_SM_aup_20150706T000000_Vv4030_001_HEGOUT.h5
DEBUG:podpac.datalib.egi:Reading file: 139973872/SMAP_L4_SM_aup_20150706T030000_Vv4030_001_HEGOUT.h5
DEBUG:podpac.datalib.egi:Reading file: 139973877/SMAP_L4_SM_aup_20150706T060000_Vv4030_001_HEGOUT.h5
DEBUG:podpac.datalib.egi:Reading file: 139973882/SMAP_L4_SM_aup_20150706T090000_Vv4030_001_HEGOUT.h5
DEBUG:podpac.datalib.egi:Reading file: 139973868/SMAP_L4_SM_aup_20150706T120000_Vv4030_001_HEGOUT.h5
DEBUG:podpac.datalib.egi:Reading file: 139973874/SMAP_L4_SM_aup_20150706T150000_Vv4030_001

In [4]:
o

<xarray.UnitsDataArray (lon: 10, lat: 10, time: 10)>
array([[[nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan],
        ...,
        [nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan]],

       [[nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan],
        ...,
        [nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan]],

       ...,

       [[nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan],
        ...,
        [nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan]],

       [[nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan],
        ...,
        [nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan]]])
Coordinates:
  * lon      (lon) float64 -82.0 -81.89 -81.78 -81.67 ... -81.22 -81.11 -81.0
  * lat      (lat) float64 38.0 38.11 38.22 38.33 ... 38.67 38.78 38.89 39.0
  * time     (time) datetime64[ns] 2015-07-06 2015-07-06T05:20:00 ... 2015-07-08
Attributes:
    layer_style:  <podpac.core.style.Style object at 

In [5]:
a = node.native_coordinates.transform('epsg:4326')

/Users/marcus/computing/creare/podpac/podpac/podpac/core/coordinates/dependent_coordinates.py:491: UserWarning: transformation of coordinate segment lengths not yet implemented
  warnings.warn("transformation of coordinate segment lengths not yet implemented")
/Users/marcus/computing/creare/podpac/podpac/podpac/core/coordinates/dependent_coordinates.py:493: UserWarning: transformation of coordinate segment lengths not yet implemented
  warnings.warn("transformation of coordinate segment lengths not yet implemented")


In [8]:
a['lat'].coordinates

array([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]])

In [8]:
node.native_coordinates.transform(crs="epsg:4326")

/Users/marcus/computing/creare/podpac/podpac/podpac/core/coordinates/dependent_coordinates.py:491: UserWarning: transformation of coordinate segment lengths not yet implemented
  warnings.warn("transformation of coordinate segment lengths not yet implemented")
/Users/marcus/computing/creare/podpac/podpac/podpac/core/coordinates/dependent_coordinates.py:493: UserWarning: transformation of coordinate segment lengths not yet implemented
  warnings.warn("transformation of coordinate segment lengths not yet implemented")
/anaconda3/envs/podpac/lib/python3.7/site-packages/numpy/core/fromnumeric.py:83: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


Coordinates(epsg:4326)
	time: ArrayCoordinates1d(time): Bounds[2015-07-05T13:00:00.000000000, 2015-07-06T16:00:00.000000000], N[10], ctype['point']
	lat,lon[lat]: DependentCoordinates(i,j->lat): Bounds[nan, nan], shape(11, 11), ctype[midpoint]
	lat,lon[lon]: DependentCoordinates(i,j->lon): Bounds[46.820542, 47.754152], shape(11, 11), ctype[midpoint]